In [1]:
import os
import yaml
from pathlib import Path
from PIL import Image
from scipy import linalg

import numpy as np
import skimage
from matplotlib import pyplot as plt
from openslide import OpenSlide

from src.utils import *

In [2]:
config = load_config()
config

{'dataset_path': '/Users/cmkim/workspace/datasets/TIGER/'}

In [3]:
dataset_path = Path(config["dataset_path"])
dataset_path

PosixPath('/Users/cmkim/workspace/datasets/TIGER')

In [4]:
# subset_path = dataset_path / "wsirois/wsi-level-annotations/images"
subset_path = dataset_path / "wsibulk/images"

subset_path

PosixPath('/Users/cmkim/workspace/datasets/TIGER/wsibulk/images')

In [5]:
# wsi_path = subset_path / os.listdir(subset_path)[0]
# wsi_path = subset_path / "111S.tif"
# wsi_path = subset_path / "103S.tif"
wsi_path = subset_path / "119S.tif"

wsi_path

PosixPath('/Users/cmkim/workspace/datasets/TIGER/wsibulk/images/119S.tif')

In [6]:
slide = OpenSlide(filename=wsi_path)
slide

OpenSlide(PosixPath('/Users/cmkim/workspace/datasets/TIGER/wsibulk/images/119S.tif'))

In [7]:
level_d = slide.level_dimensions

In [8]:
level = slide.get_best_level_for_downsample(32)

In [9]:
rgb = np.array(
    slide.read_region(
        location=(0, 0),
        level=level, 
        size=level_d[level]
    )
)

In [10]:
if rgb.shape[-1] == 4:
    rgb = rgba2rgb(rgb)

## optical density

In [11]:
transmission = skimage.util.img_as_float(rgb, force_copy=True)
np.maximum(transmission, 1e-6, out=transmission)

array([[[0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        ...,
        [0.90980392, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922]],

       [[0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        ...,
        [0.90980392, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922]],

       [[0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        [0.90980392, 0.89019608, 0.90588235],
        ...,
        [0.90980392, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922],
        [0.90196078, 0.89411765, 0.89803922]],

       ...,

       [[0.91764706, 0.90196078, 0.90588235],
        [0.91764706, 0.90196078, 0.90588235],
        [0.91764706, 0

In [12]:
od = np.log(transmission) / np.log(1e-6)
od

array([[[0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        ...,
        [0.00684203, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412]],

       [[0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        ...,
        [0.00684203, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412]],

       [[0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        [0.00684203, 0.00841905, 0.0071547 ],
        ...,
        [0.00684203, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412],
        [0.00746872, 0.00810089, 0.00778412]],

       ...,

       [[0.00622072, 0.00746872, 0.0071547 ],
        [0.00622072, 0.00746872, 0.0071547 ],
        [0.00622072, 0

## color deconvolution

In [ ]:
stain_matrix = StainConcentrationMatrix.hed()

In [ ]:
stains = od @ linalg.inv(stain_matrix)
# np.maximum(stains, 0, out=stains)